In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"

loader = PyMuPDFLoader(pdf_path, extract_images=True)

docs = loader.load()

# 문서 정보 출력
for idx, doc in enumerate(docs):
    print(f"Page {idx + 1}:")
    print("Text:", doc.page_content)  # 페이지의 텍스트 출력
    if "images" in doc.metadata:
        print("Images:", doc.metadata["images"])  # 이미지 정보 출력
    print("-" * 50)


Page 1:
Text: 딥러닝 기반 아동 미술 심리 진단 
김지민, 박형빈, 정재식
DeepPrint
Team D.P.
Wanted PotenUp
3rd Project
2025.03.05.
--------------------------------------------------
Page 2:
Text: 오늘의 발표 내용은?
01. 프로젝트 개요
02. Project Flow
03. 서비스 구현
--------------------------------------------------
Page 3:
Text: HTP 검사란?
주제 소개
House(집), Tree(나무), Person(사람)의 세
가지 그림을 그리게 하여 개인의 성격, 정서 상
태, 대인 관계 등을 평가하는 투사적 심리 검사
특히 아동 및 청소년의 심리 상태를 파악하거나,
성인의 무의식적 감정과 스트레스를 이해하는
데 유용하게 사용
01. 프로젝트 개요




HTP
田田
HTP
--------------------------------------------------
Page 4:
Text: HTP 검사의 한계
주제 선정 배경
주관적 해석의 가능성
표준화 부족
문화적 차이
피검사자의 의도적 왜곡
기술적 한계
01. 프로젝트 개요




HTPTESTDRAWING
AISGRXNONT
田田
HOUSE-TREE - PERSON
--------------------------------------------------
Page 5:
Text: Project Work Flow
학습 데이터 수집 및 정제
1
2
3
모델 학습 및 평가
결과 해석
AI Hub
: AI 기반 아동 미술심리 진단을 위
한 그림 데이터 구축
Ultralytics YOLO
: yolo11n, yolo11s, ...
Detection 모델 활용
02. Project Flow
해석 가능한 요소
: 그려진 그림 (탐지된 개체), 그림
의 상대적 크기, 그림의 위치 등.
------------------------------------

In [2]:
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# PDF 파일 로드 및 페이지 분할
loader = PyPDFLoader("C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf")  # PDF 파일 경로 설정
pages = loader.load_and_split()  # 페이지별로 텍스트 추출

# **2. 벡터 스토어 생성**
vectorstore = Chroma.from_documents(pages, embedding=OpenAIEmbeddings())  # 임베딩 생성 및 저장
retriever = vectorstore.as_retriever()  # 검색기 생성

# **3. 검색된 문서 데이터를 문자열로 변환하는 함수**
def format_context(docs):
    """검색된 문서 리스트를 하나의 문자열로 병합"""
    return "\n".join([doc.page_content for doc in docs])

# **4. 프롬프트 템플릿 정의**
template = '''다음 컨텍스트를 기반으로 발표 대본을 작성하세요:
{context}

질문: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# **5. LLM 모델 초기화**
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# **6. RAG 체인 구성**
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}  # 컨텍스트와 질문 연결
    | prompt  # 프롬프트 적용
    | model   # 언어 모델 실행
    | StrOutputParser()  # 결과 파싱
)

# **7. 체인 실행**
retrieved_docs = retriever.get_relevant_documents("이 문서를 기반으로 발표 대본을 작성해주세요.")  # 문서 검색
formatted_context = format_context(retrieved_docs)  # 검색된 문서를 문자열로 변환

result = rag_chain.invoke({
    "context": formatted_context,  # 문자열 형태의 context 전달
    "question": "페이지별로 발표 대본을 작성해주세요."
})

# 결과 출력
print(result)


C:\Users\user\AppData\Local\Temp\ipykernel_19036\3440955430.py:42: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents("이 문서를 기반으로 발표 대본을 작성해주세요.")  # 문서 검색


### 발표 대본

---

#### 01. 프로젝트 개요

안녕하세요, 오늘 발표할 주제는 HTP 검사와 그 한계에 대한 것입니다. HTP 검사는 개인의 심리적 상태를 이해하기 위한 도구로 널리 사용되고 있지만, 여러 가지 한계가 존재합니다. 오늘은 이 한계들을 살펴보고, 이를 해결하기 위한 방안을 모색해보겠습니다.

---

#### 02. HTP 검사의 한계

HTP 검사의 한계는 크게 다섯 가지로 나눌 수 있습니다. 첫 번째는 주관적 해석의 가능성입니다. 검사 결과는 해석자의 주관에 따라 달라질 수 있어, 일관된 결과를 도출하기 어렵습니다. 두 번째는 표준화 부족입니다. HTP 검사는 표준화된 절차가 부족하여, 다양한 환경에서 일관된 결과를 얻기 힘듭니다.

세 번째는 문화적 차이입니다. HTP 검사는 문화적 배경에 따라 다르게 해석될 수 있어, 다양한 문화적 맥락을 고려하지 않으면 정확한 해석이 어렵습니다. 네 번째는 피검사자의 의도적 왜곡입니다. 피검사자가 자신의 의도를 숨기거나 왜곡할 경우, 검사 결과는 신뢰성을 잃게 됩니다. 마지막으로, 기술적 한계가 있습니다. 현재의 기술로는 HTP 검사의 모든 요소를 정확히 분석하기 어려운 점이 있습니다.

---

#### 03. 주제 선정 배경

이러한 한계를 바탕으로, 우리는 HTP 검사의 개선 방안을 모색하기 위해 이 주제를 선정하였습니다. 특히, 주관적 해석의 가능성을 줄이고, 표준화를 높이며, 문화적 차이를 고려한 새로운 접근법이 필요합니다. 또한, 피검사자의 의도적 왜곡을 방지할 수 있는 방법과 기술적 한계를 극복할 수 있는 방안도 함께 고민해야 합니다.

---

#### 04. 서비스 구현

이제 우리는 이러한 한계를 극복하기 위한 서비스 구현 방안에 대해 논의하겠습니다. 이미지 업로드 및 해석 화면을 통해, 사용자가 자신의 그림을 업로드하고, AI 기반의 해석을 제공받는 시스템을 구축할 예정입니다. 이를 통해 주관적 해석의 가능성을 줄이고, 보다 객관적인 결과를 도출할 수 있을 것입니다.



In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from PyPDF2 import PdfReader

# **1. PDF 텍스트 추출**
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"  # 각 페이지의 텍스트를 추가
    return text.strip()

# PDF 파일 경로 설정
pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

# **2. 프롬프트 템플릿 정의**
template = '''다음 PDF 내용을 기반으로 발표 대본을 작성하세요:
{context}

질문: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# **3. LLM 모델 초기화**
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# **4. 프롬프트 실행**
question = "페이지별로 발표 대본을 작성해주세요."
formatted_prompt = prompt.format(context=pdf_text, question=question)

result = model(formatted_prompt)

# 결과 출력
print(result)


C:\Users\user\AppData\Local\Temp\ipykernel_19036\953420499.py:32: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = model(formatted_prompt)


content='### 발표 대본\n\n---\n\n#### 슬라이드 1: 제목 슬라이드\n안녕하세요, 여러분. 오늘 저희는 "딥러닝 기반 아동 미술 심리 진단" 프로젝트에 대해 발표하겠습니다. 저는 김지민입니다. 함께 발표할 박형빈, 정재식입니다. 저희는 DeepPrintTeam D.P. Wanted PotenUp의 3rd Project로, 발표 날짜는 2025년 3월 5일입니다.\n\n---\n\n#### 슬라이드 2: 오늘의 발표 내용\n오늘 발표할 내용은 크게 세 가지로 나뉩니다. 첫째, 프로젝트 개요에 대해 설명드리겠습니다. 둘째, Project Flow에 대해 말씀드리겠습니다. 마지막으로, 서비스 구현에 대해 자세히 설명하겠습니다.\n\n---\n\n#### 슬라이드 3: HTP 검사란?\n먼저 HTP 검사에 대해 소개하겠습니다. HTP 검사는 House(집), Tree(나무), Person(사람)의 세 가지 그림을 그리게 하여 개인의 성격, 정서 상태, 대인 관계 등을 평가하는 투사적 심리 검사입니다. 이 검사는 특히 아동 및 청소년의 심리 상태를 파악하거나 성인의 무의식적 감정과 스트레스를 이해하는 데 유용합니다.\n\n---\n\n#### 슬라이드 4: HTP 검사의 한계\n하지만 HTP 검사에는 몇 가지 한계가 있습니다. 첫째, 주관적 해석의 가능성이 있습니다. 둘째, 표준화가 부족합니다. 셋째, 문화적 차이가 존재합니다. 마지막으로, 피검사자의 의도적 왜곡과 기술적 한계가 있습니다.\n\n---\n\n#### 슬라이드 5: 프로젝트 개요\n이러한 한계를 극복하기 위해 저희는 AI 기반 아동 미술 심리 진단 시스템을 개발하게 되었습니다. 이 시스템은 HTP 검사의 객관성을 높이고, 보다 정확한 심리 분석을 제공하는 것을 목표로 하고 있습니다.\n\n---\n\n#### 슬라이드 6: Project Work Flow\n이제 Project Flow에 대해 설명드리겠습니다. 첫 번째 단계는 학습 데이터 수집 및 정제입니다. 두 번째 단계는 모델 학

In [8]:
print(result.content)

### 발표 대본

---

#### 슬라이드 1: 제목 슬라이드
안녕하세요, 여러분. 오늘 저희는 "딥러닝 기반 아동 미술 심리 진단" 프로젝트에 대해 발표하겠습니다. 저는 김지민입니다. 함께 발표할 박형빈, 정재식입니다. 저희는 DeepPrintTeam D.P. Wanted PotenUp의 3rd Project로, 발표 날짜는 2025년 3월 5일입니다.

---

#### 슬라이드 2: 오늘의 발표 내용
오늘 발표할 내용은 크게 세 가지로 나뉩니다. 첫째, 프로젝트 개요에 대해 설명드리겠습니다. 둘째, Project Flow에 대해 말씀드리겠습니다. 마지막으로, 서비스 구현에 대해 자세히 설명하겠습니다.

---

#### 슬라이드 3: HTP 검사란?
먼저 HTP 검사에 대해 소개하겠습니다. HTP 검사는 House(집), Tree(나무), Person(사람)의 세 가지 그림을 그리게 하여 개인의 성격, 정서 상태, 대인 관계 등을 평가하는 투사적 심리 검사입니다. 이 검사는 특히 아동 및 청소년의 심리 상태를 파악하거나 성인의 무의식적 감정과 스트레스를 이해하는 데 유용합니다.

---

#### 슬라이드 4: HTP 검사의 한계
하지만 HTP 검사에는 몇 가지 한계가 있습니다. 첫째, 주관적 해석의 가능성이 있습니다. 둘째, 표준화가 부족합니다. 셋째, 문화적 차이가 존재합니다. 마지막으로, 피검사자의 의도적 왜곡과 기술적 한계가 있습니다.

---

#### 슬라이드 5: 프로젝트 개요
이러한 한계를 극복하기 위해 저희는 AI 기반 아동 미술 심리 진단 시스템을 개발하게 되었습니다. 이 시스템은 HTP 검사의 객관성을 높이고, 보다 정확한 심리 분석을 제공하는 것을 목표로 하고 있습니다.

---

#### 슬라이드 6: Project Work Flow
이제 Project Flow에 대해 설명드리겠습니다. 첫 번째 단계는 학습 데이터 수집 및 정제입니다. 두 번째 단계는 모델 학습 및 평가입니다. 마지막으로, 결과 해석 단계가 있습니다. 저희는 A

In [6]:
import yt_dlp
from pydub import AudioSegment
import os

def time_convert(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

def download_audio(url, save_dir, clip_idx, start=None, end=None):
    os.makedirs(save_dir, exist_ok=True)

    # 확장자 포함된 yt-dlp 저장용 임시 파일 경로
    temp_template = os.path.join(save_dir, f"cheo_temp_{clip_idx}.%(ext)s")
    temp_wav = os.path.join(save_dir, f"cheo_temp_{clip_idx}.wav")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': temp_template,
        'quiet': True,
        'force_ipv4': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # 자르고 저장
    audio = AudioSegment.from_file(temp_wav, format="wav")
    if start and end:
        start_ms = time_convert(start) * 1000
        end_ms = time_convert(end) * 1000
        audio = audio[start_ms:end_ms]

    # 최종 파일명: cheo_1.wav, cheo_2.wav ...
    final_path = os.path.join(save_dir, f"winter_{clip_idx}.wav")
    audio.export(final_path, format="wav")

    os.remove(temp_wav)
    return audio


In [7]:
url = "https://www.youtube.com/watch?v=4vS0Bhe5zjA"
save_dir = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"
audio_data = []
for i in range(5):
    start = f"{12+i}:00"
    end = f"{12+i+1}:00"
    saved = download_audio(url, save_dir, clip_idx=i+1, start=start, end=end)
    audio_data.append(saved)


In [1]:
from zonos.model import Zonos

print("Zonos import 성공")


c:\wanted\Lang\lang\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Zonos import 성공


In [5]:
import os

os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"

from phonemizer.backend import EspeakBackend

backend = EspeakBackend(language='ko')
print(backend.phonemize(["야 이제 되냐 안 되냐"]))

['jɐ idʑe dwenjɐ ɐn dwenjɐ ']


In [9]:
import os
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
import torch._dynamo  # suppress warning if needed

# ❗ phonemizer용 환경변수 설정 (espeak.dll 대응)
os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"

# ❗ torch compile 비활성화 (C++ 컴파일러 없이 실행)
torch._dynamo.config.suppress_errors = True

# 경로 설정
path = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"

# 모델 불러오기
device = "cuda"  # 또는 "cpu"
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)

# 입력 음성 로드
wav, sampling_rate = torchaudio.load(os.path.join(path, "cheo_1.wav"))

# 스피커 임베딩 생성 (제로샷 보이스 클로닝)
speaker = model.make_speaker_embedding(wav, sampling_rate)

# 텍스트와 스피커로 조건 딕셔너리 구성
cond_dict = make_cond_dict(text="Project고 무엇이고 다 때려 쳐 버리고 싶다", speaker=speaker, language="ko")
conditioning = model.prepare_conditioning(cond_dict)

# 음성 생성 (컴파일러 비활성화)
codes = model.generate(conditioning, disable_torch_compile=True)
wavs = model.autoencoder.decode(codes).cpu()

# 결과 저장
torchaudio.save(os.path.join(path, "sample.wav"), wavs[0], model.autoencoder.sampling_rate)

print("🔊 sample.wav 생성 완료!")


Generating:  14%|█▍        | 358/2588 [01:14<07:43,  4.81it/s]


🔊 sample.wav 생성 완료!


In [1]:
# !pip install -q kokoro>=0.9.2 soundfile
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1

������ ��θ� ã�� �� �����ϴ�.


In [2]:
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch

c:\wanted\Lang\lang\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 🇺🇸 'a' => 미국 영어, 🇬🇧 'b' => 영국 영어
# 🇯🇵 'j' => 일본어: pip install misaki[ja]
# 🇨🇳 'z' => 중국어(보통화): pip install misaki[zh]
pipeline = KPipeline(lang_code='ko') # <= lang_code가 목소리 언어와 일치하는지 확인할 것

# 이 텍스트는 데모용이며, 학습 중에는 사용되지 않았습니다

AssertionError: ('ko', {'a': 'American English', 'b': 'British English', 'e': 'es', 'f': 'fr-fr', 'h': 'hi', 'i': 'it', 'p': 'pt-br', 'j': 'Japanese', 'z': 'Mandarin Chinese'})